# MARec++: Comprehensive Metadata-Driven Cold-Start Recommendation

**Research Question**: Can richer and more comprehensive metadata representations
outperform the original MARec results in cold-start recommendation, without
changing the backbone architecture?

## Experiment Plan
1. **Baseline reproduction** — MARec with top3 features (actors, directors, genres)
2. **Tag modes** — no_tags, tags_train_users (leakage-safe), tags_full (upper bound)
3. **Feature ablation** — individual and combined feature contributions
4. **Control: random tag shuffle** — verify tags carry content signal, not density
5. **Cold-item tag coverage** — analyze how many cold items have tags
6. **Publication-quality tables & plots**

All runs use **10 splits** with bootstrap CIs and paired significance tests.

## 0. Setup & Installation

In [ ]:
import subprocess, sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                       "numpy", "scipy", "pandas", "scikit-learn", "matplotlib",
                       "seaborn", "bottleneck", "tabulate"])

In [ ]:
import os, time, json, warnings, copy, zipfile, urllib.request
from pathlib import Path
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Set, Tuple
from collections import Counter
from itertools import product as iterproduct

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.linear_model import LinearRegression
from scipy.optimize import nnls

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({
    "font.size": 12, "axes.titlesize": 14, "axes.labelsize": 12,
    "xtick.labelsize": 10, "ytick.labelsize": 10,
    "figure.figsize": (10, 6), "figure.dpi": 120,
})
try:
    import seaborn as sns
    sns.set_style("whitegrid")
except ImportError:
    pass

try:
    import bottleneck as bn
    _USE_BN = True
except ImportError:
    _USE_BN = False

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
print("All imports successful")

## 1. Data Loading (with user-tags fix)

In [ ]:
HETREC_URL = "http://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip"

def download_hetrec(data_dir):
    data_dir = Path(data_dir)
    data_dir.mkdir(parents=True, exist_ok=True)
    raw_dir = data_dir / "hetrec2011-movielens-2k-v2"
    if raw_dir.exists():
        return raw_dir
    ratings_file = data_dir / "user_ratedmovies-timestamps.dat"
    if ratings_file.exists():
        return data_dir
    zip_path = data_dir / "hetrec2011-movielens-2k-v2.zip"
    print(f"Downloading HetRec 2011 to {zip_path}...")
    urllib.request.urlretrieve(HETREC_URL, str(zip_path))
    with zipfile.ZipFile(str(zip_path), "r") as zf:
        zf.extractall(str(data_dir))
    if raw_dir.exists():
        return raw_dir
    elif ratings_file.exists():
        return data_dir
    raise RuntimeError("Extraction failed")

def read_dat(raw_dir, filename):
    path = Path(raw_dir) / filename
    if not path.exists():
        return pd.DataFrame()
    with open(str(path), "r", encoding="latin-1") as f:
        text = f.read()
    lines = text.strip().split("\n")
    rows = [line.replace("\r", "").split("\t") for line in lines]
    return pd.DataFrame(rows[1:], columns=rows[0])

def load_all_dataframes(raw_dir):
    dfs = {}
    dfs["movies"] = read_dat(raw_dir, "movies.dat")
    dfs["ratings"] = read_dat(raw_dir, "user_ratedmovies-timestamps.dat")
    dfs["genres"] = read_dat(raw_dir, "movie_genres.dat")
    dfs["actors"] = read_dat(raw_dir, "movie_actors.dat")
    dfs["directors"] = read_dat(raw_dir, "movie_directors.dat")
    dfs["countries"] = read_dat(raw_dir, "movie_countries.dat")
    dfs["tags"] = read_dat(raw_dir, "movie_tags.dat")
    # FIX: Load per-user tag assignments (has userID column for leakage-safe modes)
    dfs["user_tags"] = read_dat(raw_dir, "user_taggedmovies-timestamps.dat")
    if dfs["user_tags"].empty:
        dfs["user_tags"] = read_dat(raw_dir, "user_taggedmovies.dat")
    dfs["locations"] = read_dat(raw_dir, "movie_locations.dat")
    return dfs

def binarize_and_kcore(df_ratings, threshold=3.0, min_user=5, min_item=5):
    df = df_ratings.copy()
    df["userID"] = df["userID"].astype(str)
    df["movieID"] = df["movieID"].astype(str)
    df["rating"] = df["rating"].astype(float)
    df = df[df["rating"] >= threshold].copy()
    df = df.groupby(["userID", "movieID"])["rating"].max().reset_index()
    for _ in range(200):
        n = len(df)
        ic = df["movieID"].value_counts()
        df = df[df["movieID"].isin(ic[ic >= min_item].index)]
        uc = df["userID"].value_counts()
        df = df[df["userID"].isin(uc[uc >= min_user].index)]
        if len(df) == n:
            break
    users = sorted(df["userID"].unique())
    items = sorted(df["movieID"].unique())
    user2idx = {u: i for i, u in enumerate(users)}
    item2idx = {m: i for i, m in enumerate(items)}
    idx2item = {i: m for m, i in item2idx.items()}
    rows = np.array([user2idx[u] for u in df["userID"]])
    cols = np.array([item2idx[m] for m in df["movieID"]])
    URM = sp.csr_matrix(
        (np.ones(len(rows), dtype=np.float32), (rows, cols)),
        shape=(len(users), len(items)))
    return URM, user2idx, item2idx, idx2item, df

def load_dataset(data_dir="hetrec_data", threshold=3.0):
    raw_dir = download_hetrec(data_dir)
    dfs = load_all_dataframes(raw_dir)
    URM, user2idx, item2idx, idx2item, df_filtered = binarize_and_kcore(
        dfs["ratings"], threshold=threshold)
    n_u, n_i = URM.shape
    density = URM.nnz / (n_u * n_i) * 100
    print(f"Dataset: {n_u} users x {n_i} items, {URM.nnz} interactions, density={density:.2f}%")
    return {"URM": URM, "user2idx": user2idx, "item2idx": item2idx,
            "idx2item": idx2item, "df_filtered": df_filtered, "dfs": dfs,
            "n_users": n_u, "n_items": n_i}

print("Data loading functions defined")

## 2. Feature Encoding (with tag-mode fix)

In [ ]:
def _build_feature_dict(df, id_col, val_col):
    d = {}
    for _, row in df.iterrows():
        d.setdefault(str(row[id_col]), []).append(str(row[val_col]))
    return d

def encode_multihot(item_lists, min_count=2):
    flat = [v for lst in item_lists for v in lst]
    counter = Counter(flat)
    valid = sorted(k for k, v in counter.items() if v >= min_count and k.strip())
    if not valid:
        return np.zeros((len(item_lists), 1), dtype=np.float32)
    mlb = MultiLabelBinarizer(classes=valid, sparse_output=False)
    return mlb.fit_transform(item_lists).astype(np.float32)

def encode_tfidf(texts, max_features=10000):
    if not any(t.strip() for t in texts):
        return np.zeros((len(texts), 1), dtype=np.float32)
    try:
        pipe = Pipeline([("count", CountVectorizer(max_features=max_features)),
                         ("tfidf", TfidfTransformer())])
        return pipe.fit_transform(texts).toarray().astype(np.float32)
    except ValueError:
        return np.zeros((len(texts), 1), dtype=np.float32)

def encode_years(year_values):
    arr = np.array(year_values, dtype=np.float32).reshape(-1, 1)
    return MinMaxScaler().fit_transform(arr)

def _filter_tags_train_pairs(df_tags, train_csr, user2idx, item2idx):
    result = {}
    for _, row in df_tags.iterrows():
        uid = str(row.get("userID", ""))
        mid = str(row.get("movieID", ""))
        tid = str(row.get("tagID", ""))
        if uid not in user2idx or mid not in item2idx:
            continue
        u_idx, i_idx = user2idx[uid], item2idx[mid]
        if train_csr[u_idx, i_idx] > 0:
            result.setdefault(mid, []).append(tid)
    return result

def _filter_tags_train_users(df_tags, train_csr, user2idx, item2idx):
    train_users = set(train_csr.tocoo().row)
    result = {}
    for _, row in df_tags.iterrows():
        uid = str(row.get("userID", ""))
        mid = str(row.get("movieID", ""))
        tid = str(row.get("tagID", ""))
        if uid not in user2idx or mid not in item2idx:
            continue
        u_idx = user2idx[uid]
        if u_idx in train_users:
            result.setdefault(mid, []).append(tid)
    return result

def build_feature_matrices(dfs, item2idx, idx2item, n_items,
                           tag_mode="no_tags", train_csr=None,
                           user2idx=None, min_count=2,
                           shuffle_tags=False):
    assert tag_mode in ("no_tags", "tags_train_only", "tags_train_users",
                        "tags_train_pairs", "tags_full")
    if tag_mode in ("tags_train_only", "tags_train_users", "tags_train_pairs"):
        assert train_csr is not None and user2idx is not None

    dict_genres = _build_feature_dict(dfs["genres"], "movieID", "genre")
    dict_actors = _build_feature_dict(dfs["actors"], "movieID", "actorName")
    dict_directors = _build_feature_dict(dfs["directors"], "movieID", "directorName")
    dict_countries = _build_feature_dict(dfs["countries"], "movieID", "country")

    # FIX: Use dfs["user_tags"] for leakage-safe modes
    if tag_mode == "tags_full":
        dict_tags = _build_feature_dict(dfs["tags"], "movieID", "tagID")
    elif tag_mode == "tags_train_pairs":
        dict_tags = _filter_tags_train_pairs(
            dfs.get("user_tags", dfs["tags"]), train_csr, user2idx, item2idx)
    elif tag_mode == "tags_train_users":
        dict_tags = _filter_tags_train_users(
            dfs.get("user_tags", dfs["tags"]), train_csr, user2idx, item2idx)
    elif tag_mode == "tags_train_only":
        dict_tags = _filter_tags_train_pairs(
            dfs.get("user_tags", dfs["tags"]), train_csr, user2idx, item2idx)
    else:
        dict_tags = {}

    df_movies = dfs["movies"]
    dict_years = {}
    for _, row in df_movies.iterrows():
        mid = str(row["id"])
        try:
            dict_years[mid] = int(row["year"])
        except (ValueError, KeyError):
            dict_years[mid] = 2000

    dict_locs = [{}, {}, {}]
    if len(dfs["locations"]) > 0:
        lc = dfs["locations"].columns.tolist()
        for _, row in dfs["locations"].iterrows():
            mid = str(row[lc[0]])
            for lvl in range(min(3, len(lc) - 1)):
                dict_locs[lvl].setdefault(mid, []).append(str(row[lc[lvl + 1]]))

    feat_lists = {"genres": [], "actors": [], "directors": [], "countries": [],
                  "loc1": [], "loc2": [], "loc3": [], "years": []}
    loc_texts = []
    for i in range(n_items):
        mid = str(idx2item[i])
        feat_lists["genres"].append(dict_genres.get(mid, []))
        feat_lists["actors"].append(dict_actors.get(mid, []))
        feat_lists["directors"].append(dict_directors.get(mid, []))
        feat_lists["countries"].append(dict_countries.get(mid, []))
        feat_lists["loc1"].append(dict_locs[0].get(mid, []))
        feat_lists["loc2"].append(dict_locs[1].get(mid, []))
        feat_lists["loc3"].append(dict_locs[2].get(mid, []))
        feat_lists["years"].append(dict_years.get(mid, 2000))
        loc_parts = [" ".join(dict_locs[j].get(mid, [])) for j in range(3)]
        loc_texts.append(" ".join(loc_parts).strip())

    scaler = MinMaxScaler()
    encoded = {}
    for name in ["genres", "actors", "directors", "countries", "loc1", "loc2", "loc3"]:
        raw = encode_multihot(feat_lists[name], min_count=min_count)
        encoded[name] = scaler.fit_transform(raw)
    encoded["locations"] = scaler.fit_transform(encode_tfidf(loc_texts, max_features=10000))
    encoded["years"] = encode_years(feat_lists["years"])

    tag_stats = None
    if tag_mode != "no_tags":
        if not dict_tags:
            encoded["tags"] = np.zeros((n_items, 1), dtype=np.float32)
            tag_stats = {"mode": tag_mode, "n_entries": 0, "n_unique_tags": 0,
                         "n_items_with_tags": 0, "avg_tags_per_item": 0.0, "n_features": 1}
        else:
            tag_lists = [dict_tags.get(str(idx2item[i]), []) for i in range(n_items)]
            if shuffle_tags:
                rng = np.random.RandomState(999)
                rng.shuffle(tag_lists)
            n_tag_entries = sum(len(t) for t in tag_lists)
            n_items_with_tags = sum(1 for t in tag_lists if len(t) > 0)
            unique_tags = set(t for tl in tag_lists for t in tl)
            raw_tags = encode_multihot(tag_lists, min_count=min_count)
            encoded["tags"] = scaler.fit_transform(raw_tags)
            print(f"  Tags ({tag_mode}{'|SHUFFLED' if shuffle_tags else ''}): "
                  f"{n_tag_entries} entries, {len(unique_tags)} unique, "
                  f"{n_items_with_tags} items, {encoded['tags'].shape[1]} features")
            tag_stats = {"mode": tag_mode, "n_entries": n_tag_entries,
                         "n_unique_tags": len(unique_tags),
                         "n_items_with_tags": n_items_with_tags,
                         "n_features": encoded["tags"].shape[1]}

    if tag_stats:
        encoded["_tag_stats"] = tag_stats
    return encoded

print("Feature encoding functions defined")

## 3. Similarity, Splits, MARec Core

In [ ]:
def smoothed_cosine_similarity(enc, shrinkage=20.0):
    sim_num = enc @ enc.T
    norms = np.linalg.norm(enc, axis=1)
    sim_den = np.outer(norms, norms) + shrinkage
    sim = sim_num / sim_den
    np.fill_diagonal(sim, 0.0)
    return sim

def year_similarity(enc):
    dist = euclidean_distances(enc)
    max_dist = dist.max()
    sim = 1.0 - dist / (max_dist + 1e-10) if max_dist > 0 else np.zeros_like(dist)
    np.fill_diagonal(sim, 0.0)
    return sim

def build_similarity_matrices(encoded_features, shrinkage=20.0,
                               per_feature_shrinkage=None, cross_pairs=None):
    S = {}
    for name, enc in encoded_features.items():
        if name.startswith("_"):
            continue
        if name == "years":
            S[name] = year_similarity(enc)
        else:
            delta = (per_feature_shrinkage or {}).get(name, shrinkage)
            S[name] = smoothed_cosine_similarity(enc, shrinkage=delta)
    if cross_pairs:
        for a, b in cross_pairs:
            if a in S and b in S:
                S[f"{a}_x_{b}"] = S[a] * S[b]
    return S

def create_cold_split_sparse(URM, cold_frac=0.20, seed=42):
    rng = np.random.RandomState(seed)
    n_u, n_i = URM.shape
    n_cold = int(n_i * cold_frac)
    cold_items = np.sort(rng.choice(n_i, n_cold, replace=False))
    cold_set = set(cold_items)
    coo = URM.tocoo()
    train_mask = np.array([c not in cold_set for c in coo.col])
    test_mask = ~train_mask
    train = sp.csr_matrix((coo.data[train_mask], (coo.row[train_mask], coo.col[train_mask])),
                          shape=(n_u, n_i))
    test = sp.csr_matrix((coo.data[test_mask], (coo.row[test_mask], coo.col[test_mask])),
                         shape=(n_u, n_i))
    return train, test, cold_items

def generate_splits(URM, n_splits=10, cold_frac=0.20, base_seed=42):
    return [{"train": t, "test": te, "cold_items": ci, "seed": base_seed + i}
            for i, (t, te, ci) in enumerate(
                [create_cold_split_sparse(URM, cold_frac, base_seed + i)
                 for i in range(n_splits)])]

def compute_dr(X, beta, percentile=10):
    v = np.asarray(X.sum(axis=0)).ravel() if sp.issparse(X) else X.sum(axis=0)
    p = max(np.percentile(v, percentile), 1.0)
    k = beta / p
    return np.where(v <= p, k * (p - v), 0.0)

def ease_aligned(X, Xtilde, lambda1=1.0, beta=1.0, alpha=1.0,
                 dr_percentile=10, XtX=None):
    n = X.shape[1]
    dr = compute_dr(X, beta, percentile=dr_percentile)
    Xt_dr = (alpha * Xtilde) * dr[np.newaxis, :]
    XtXt_IR = X.T @ Xt_dr
    if XtX is None:
        XtX = X.T @ X
    P = np.linalg.inv(XtX + lambda1 * np.eye(n) + XtXt_IR)
    Bt = P @ (XtX + XtXt_IR)
    gamma = np.diag(Bt) / np.diag(P)
    B = Bt - P @ np.diag(gamma)
    return B

print("Similarity, splits, and MARec core defined")

## 4. Evaluation Metrics

In [ ]:
def _argpartition_topk(arr, k, axis=1):
    if _USE_BN:
        return bn.argpartition(-arr, k, axis=axis)
    return np.argpartition(-arr, k, axis=axis)

def hit_rate_at_k(X_pred, heldout, k):
    n = X_pred.shape[0]
    if k >= X_pred.shape[1]:
        k = X_pred.shape[1] - 1
    idx = _argpartition_topk(X_pred, k)
    pred_bin = np.zeros_like(X_pred, dtype=bool)
    pred_bin[np.arange(n)[:, np.newaxis], idx[:, :k]] = True
    true_bin = (heldout > 0).toarray() if sp.issparse(heldout) else (heldout > 0)
    hits = np.logical_and(true_bin, pred_bin).sum(axis=1).astype(np.float64)
    n_rel = true_bin.sum(axis=1).astype(np.float64)
    denom = np.minimum(k, n_rel)
    return hits / np.maximum(denom, 1.0)

def ndcg_at_k(X_pred, heldout, k):
    n = X_pred.shape[0]
    if k >= X_pred.shape[1]:
        k = X_pred.shape[1] - 1
    idx_part = _argpartition_topk(X_pred, k)
    topk_part = X_pred[np.arange(n)[:, np.newaxis], idx_part[:, :k]]
    idx_sort = np.argsort(-topk_part, axis=1)
    idx_topk = idx_part[np.arange(n)[:, np.newaxis], idx_sort]
    tp = 1.0 / np.log2(np.arange(2, k + 2))
    heldout_arr = heldout.toarray() if sp.issparse(heldout) else heldout
    DCG = (heldout_arr[np.arange(n)[:, np.newaxis], idx_topk] * tp).sum(axis=1)
    n_rel = (heldout_arr > 0).sum(axis=1)
    IDCG = np.array([tp[:min(int(nr), k)].sum() for nr in n_rel])
    return DCG / np.maximum(IDCG, 1e-10)

def evaluate(pred, test_data, train_data, ks=(10, 25), cold_items=None):
    pred = pred.copy()
    n_items = pred.shape[1]
    train_coo = train_data.tocoo()
    pred[train_coo.row, train_coo.col] = -np.inf
    if cold_items is not None:
        warm_items = np.setdiff1d(np.arange(n_items), cold_items)
        pred[:, warm_items] = -np.inf
    test_users = np.where(np.asarray(test_data.sum(axis=1)).ravel() > 0)[0]
    if len(test_users) == 0:
        return {f"{m}@{k}": 0.0 for k in ks for m in ("hr", "ndcg")}
    p, t = pred[test_users], test_data[test_users]
    results = {}
    for k in ks:
        if k >= p.shape[1]:
            continue
        results[f"hr@{k}"] = float(np.nanmean(hit_rate_at_k(p, t, k)))
        results[f"ndcg@{k}"] = float(np.nanmean(ndcg_at_k(p, t, k)))
    return results

def paired_ttest(scores_a, scores_b):
    from scipy import stats
    t_stat, p_value = stats.ttest_rel(scores_a, scores_b)
    return float(t_stat), float(p_value)

print("Evaluation metrics defined")

## 5. Experiment Runner

In [ ]:
def run_single_prediction(train_csr, feat_names, ds, tag_mode, hp,
                          min_count=2, shrinkage=20.0, shuffle_tags=False):
    needs_tags = any("tags" in f for f in feat_names)
    actual_tag_mode = tag_mode if needs_tags else "no_tags"
    encoded = build_feature_matrices(
        ds["dfs"], ds["item2idx"], ds["idx2item"], ds["n_items"],
        tag_mode=actual_tag_mode,
        train_csr=train_csr if actual_tag_mode not in ("no_tags", "tags_full") else None,
        user2idx=ds["user2idx"] if actual_tag_mode not in ("no_tags", "tags_full") else None,
        min_count=min_count, shuffle_tags=shuffle_tags)
    tag_stats = encoded.pop("_tag_stats", None)
    if needs_tags:
        assert "tags" in encoded, f"Tags required but not built! tag_mode={actual_tag_mode}"
    S = build_similarity_matrices(encoded, shrinkage=shrinkage)
    sim_list = []
    for f in feat_names:
        if f in S:
            sim_list.append(S[f])
        else:
            for sname in S:
                if f in sname:
                    sim_list.append(S[sname])
                    break
    X = train_csr.toarray().astype(np.float64)
    XtX = X.T @ X
    XG = [X @ G for G in sim_list]
    y = X.ravel()
    Xr = np.column_stack([xg.ravel() for xg in XG])
    w = np.where(y > 0, 1.0, 0.2)
    reg = LinearRegression(fit_intercept=True)
    reg.fit(Xr, y, sample_weight=w)
    coefs = reg.coef_
    Xtilde = sum(c * xg for c, xg in zip(coefs, XG))
    B = ease_aligned(X, Xtilde, lambda1=hp["lambda1"], beta=hp["beta"],
                     alpha=hp["alpha"], XtX=XtX)
    return X @ B, tag_stats

def run_experiment(ds, splits, feat_names, tag_mode, config_name,
                   hp_grid=None, n_splits=10, eval_ks=(10, 25),
                   min_count=2, shrinkage=20.0, tune_splits=None,
                   shuffle_tags=False):
    if hp_grid is None:
        hp_grid = {"lambda1": [0.1, 1, 10, 100],
                   "beta": [1, 10, 100, 500],
                   "alpha": [0.1, 1, 10, 100]}
    if tune_splits is None:
        tune_splits = [0, 1, 2]
    n_splits = min(n_splits, len(splits))
    print(f"\n{'='*65}")
    print(f"  Experiment: {config_name}")
    print(f"  Features: {feat_names}")
    print(f"  Tag mode: {tag_mode}{'  [SHUFFLED CONTROL]' if shuffle_tags else ''}")
    print(f"  Splits: {n_splits}, Tune on: {tune_splits}")
    print(f"{'='*65}")
    # HP Tuning
    t0 = time.time()
    best_score, best_hp = -1, {"lambda1": 1.0, "beta": 100.0, "alpha": 1.0}
    hp_combos = list(iterproduct(hp_grid["lambda1"], hp_grid["beta"], hp_grid["alpha"]))
    print(f"  HP grid: {len(hp_combos)} combos")
    for l1, beta, alpha in hp_combos:
        scores = []
        for si in tune_splits:
            if si >= len(splits):
                continue
            pred, _ = run_single_prediction(
                splits[si]["train"], feat_names, ds, tag_mode,
                {"lambda1": l1, "beta": beta, "alpha": alpha},
                min_count=min_count, shrinkage=shrinkage, shuffle_tags=shuffle_tags)
            m = evaluate(pred, splits[si]["test"], splits[si]["train"],
                         ks=[eval_ks[0]], cold_items=splits[si]["cold_items"])
            scores.append(m.get(f"hr@{eval_ks[0]}", 0))
        avg = np.mean(scores) if scores else 0
        if avg > best_score:
            best_score = avg
            best_hp = {"lambda1": l1, "beta": beta, "alpha": alpha}
    print(f"  Best HP: l1={best_hp['lambda1']}, beta={best_hp['beta']}, "
          f"alpha={best_hp['alpha']} (tune={best_score:.4f}, {time.time()-t0:.0f}s)")
    # Full Evaluation
    t1 = time.time()
    per_split = []
    tag_stats = None
    for si in range(n_splits):
        pred, ts = run_single_prediction(
            splits[si]["train"], feat_names, ds, tag_mode, best_hp,
            min_count=min_count, shrinkage=shrinkage, shuffle_tags=shuffle_tags)
        if ts and tag_stats is None:
            tag_stats = ts
        m = evaluate(pred, splits[si]["test"], splits[si]["train"],
                     ks=eval_ks, cold_items=splits[si]["cold_items"])
        per_split.append(m)
        if si == 0:
            print(f"  Split 0: {m}")
    avg = {k: np.mean([m[k] for m in per_split]) for k in per_split[0]}
    std = {k: np.std([m[k] for m in per_split]) for k in per_split[0]}
    print(f"\n  Results ({n_splits} splits, {time.time()-t1:.0f}s):")
    for k in sorted(avg.keys()):
        print(f"     {k}: {avg[k]:.4f} +/- {std[k]:.4f}")
    return {"config_name": config_name, "feature_names": feat_names,
            "tag_mode": tag_mode, "best_hp": best_hp,
            "avg_metrics": avg, "std_metrics": std,
            "per_split_metrics": per_split, "tag_stats": tag_stats,
            "shuffle_tags": shuffle_tags}

print("Experiment runner defined")

## 6. Load Data & Generate Splits

In [ ]:
print("Loading dataset...")
ds = load_dataset("hetrec_data", threshold=3.0)

N_SPLITS = 10
print(f"\nGenerating {N_SPLITS} cold-start splits...")
splits = generate_splits(ds["URM"], n_splits=N_SPLITS, cold_frac=0.20, base_seed=42)
print(f"{N_SPLITS} splits generated")

# Verify split integrity
s0 = splits[0]
cold_set = set(s0["cold_items"])
train_cols = set(s0["train"].tocoo().col)
test_cols = set(s0["test"].tocoo().col)
assert len(train_cols & cold_set) == 0, "LEAK: cold items in train!"
assert test_cols.issubset(cold_set), "LEAK: warm items in test!"
print(f"Split integrity verified (cold items: {len(cold_set)})")

## 7. Tag Statistics

In [ ]:
print("\n" + "="*65)
print("  TAG STATISTICS")
print("="*65)
tag_df = ds["dfs"]["tags"]
user_tag_df = ds["dfs"]["user_tags"]
print(f"  movie_tags.dat: {len(tag_df)} rows, columns: {tag_df.columns.tolist()}")
print(f"  user_taggedmovies: {len(user_tag_df)} rows, columns: {user_tag_df.columns.tolist()}")
unique_tags_agg = tag_df["tagID"].nunique() if "tagID" in tag_df.columns else 0
unique_items_tagged = tag_df["movieID"].nunique() if "movieID" in tag_df.columns else 0
unique_users_tagging = user_tag_df["userID"].nunique() if "userID" in user_tag_df.columns else 0
print(f"  Unique tag IDs: {unique_tags_agg}")
print(f"  Items with tags (aggregated): {unique_items_tagged}")
print(f"  Users who tagged: {unique_users_tagging}")
print(f"  Items in dataset: {ds['n_items']}")
print(f"  Tag coverage: {unique_items_tagged/ds['n_items']*100:.1f}% of items have tags")

cold_items_0 = splits[0]["cold_items"]
cold_item_ids = set(str(ds["idx2item"][i]) for i in cold_items_0)
tagged_items = set(tag_df["movieID"].astype(str).unique())
cold_with_tags = cold_item_ids & tagged_items
print(f"\n  Cold items (split 0): {len(cold_items_0)}")
print(f"  Cold items with tags: {len(cold_with_tags)} ({len(cold_with_tags)/len(cold_items_0)*100:.1f}%)")

## 8. Experiment 1: Baseline Reproduction (10 splits)

In [ ]:
HP_GRID = {
    "lambda1": [0.1, 1, 10, 100],
    "beta": [1, 10, 100, 500],
    "alpha": [0.1, 1, 10, 100],
}
TUNE_SPLITS = [0, 1, 2]

results = {}

results["top3_no_tags"] = run_experiment(
    ds, splits, feat_names=["actors", "directors", "genres"],
    tag_mode="no_tags", config_name="top3_no_tags",
    hp_grid=HP_GRID, n_splits=N_SPLITS, tune_splits=TUNE_SPLITS)

## 9. Experiment 2: Tag Modes (10 splits)

In [ ]:
# Tags train users (leakage-safe)
results["top3_tags_safe"] = run_experiment(
    ds, splits, feat_names=["actors", "directors", "genres", "tags"],
    tag_mode="tags_train_users", config_name="top3_tags_safe",
    hp_grid=HP_GRID, n_splits=N_SPLITS, tune_splits=TUNE_SPLITS)

In [ ]:
# Tags full (upper bound)
results["top3_tags_full"] = run_experiment(
    ds, splits, feat_names=["actors", "directors", "genres", "tags"],
    tag_mode="tags_full", config_name="top3_tags_full",
    hp_grid=HP_GRID, n_splits=N_SPLITS, tune_splits=TUNE_SPLITS)

## 10. Experiment 3: Feature Ablation (10 splits)

In [ ]:
# Individual features
for feat_name in ["actors", "directors", "genres"]:
    results[f"single_{feat_name}"] = run_experiment(
        ds, splits, feat_names=[feat_name],
        tag_mode="no_tags", config_name=f"single_{feat_name}",
        hp_grid=HP_GRID, n_splits=N_SPLITS, tune_splits=TUNE_SPLITS)

In [ ]:
# Tags only (leakage-safe)
results["single_tags_safe"] = run_experiment(
    ds, splits, feat_names=["tags"],
    tag_mode="tags_train_users", config_name="single_tags_safe",
    hp_grid=HP_GRID, n_splits=N_SPLITS, tune_splits=TUNE_SPLITS)

In [ ]:
# Top3 + countries
results["top3_countries"] = run_experiment(
    ds, splits, feat_names=["actors", "directors", "genres", "countries"],
    tag_mode="no_tags", config_name="top3_countries",
    hp_grid=HP_GRID, n_splits=N_SPLITS, tune_splits=TUNE_SPLITS)

# Top3 + locations
results["top3_locations"] = run_experiment(
    ds, splits, feat_names=["actors", "directors", "genres", "locations"],
    tag_mode="no_tags", config_name="top3_locations",
    hp_grid=HP_GRID, n_splits=N_SPLITS, tune_splits=TUNE_SPLITS)

# Top3 + years
results["top3_years"] = run_experiment(
    ds, splits, feat_names=["actors", "directors", "genres", "years"],
    tag_mode="no_tags", config_name="top3_years",
    hp_grid=HP_GRID, n_splits=N_SPLITS, tune_splits=TUNE_SPLITS)

In [ ]:
# Full 9-feature baseline
results["base9_no_tags"] = run_experiment(
    ds, splits,
    feat_names=["genres", "actors", "directors", "countries",
                "loc1", "loc2", "loc3", "years", "locations"],
    tag_mode="no_tags", config_name="base9_no_tags",
    hp_grid=HP_GRID, n_splits=N_SPLITS, tune_splits=TUNE_SPLITS)

# Full 9-feature + tags
results["base9_tags_safe"] = run_experiment(
    ds, splits,
    feat_names=["genres", "actors", "directors", "countries",
                "loc1", "loc2", "loc3", "years", "locations", "tags"],
    tag_mode="tags_train_users", config_name="base9_tags_safe",
    hp_grid=HP_GRID, n_splits=N_SPLITS, tune_splits=TUNE_SPLITS)

## 11. Control: Random Tag Shuffle

In [ ]:
# If tags carry content signal, shuffling should collapse performance
results["tags_shuffled"] = run_experiment(
    ds, splits, feat_names=["actors", "directors", "genres", "tags"],
    tag_mode="tags_train_users", config_name="tags_shuffled",
    hp_grid=HP_GRID, n_splits=N_SPLITS, tune_splits=TUNE_SPLITS,
    shuffle_tags=True)

## 12. Results Summary Table

In [ ]:
print("\n" + "="*80)
print("  COMPREHENSIVE RESULTS TABLE")
print("="*80)

display_order = [
    "single_actors", "single_directors", "single_genres",
    "top3_no_tags", "top3_countries", "top3_locations", "top3_years",
    "base9_no_tags",
    "single_tags_safe", "top3_tags_safe", "base9_tags_safe",
    "top3_tags_full",
    "tags_shuffled",
]

rows = []
baseline_hr = results.get("top3_no_tags", {}).get("avg_metrics", {}).get("hr@10", 0)

for name in display_order:
    if name not in results:
        continue
    r = results[name]
    avg = r["avg_metrics"]
    std = r["std_metrics"]
    delta = ((avg.get("hr@10", 0) / baseline_hr) - 1) * 100 if baseline_hr > 0 else 0
    rows.append({
        "Config": name,
        "Features": ", ".join(r["feature_names"]),
        "Tag Mode": r["tag_mode"],
        "hr@10": f"{avg.get('hr@10', 0):.4f} +/- {std.get('hr@10', 0):.4f}",
        "ndcg@10": f"{avg.get('ndcg@10', 0):.4f} +/- {std.get('ndcg@10', 0):.4f}",
        "Delta hr@10": f"{delta:+.1f}%",
    })

df_results = pd.DataFrame(rows)
try:
    from tabulate import tabulate
    print(tabulate(df_results, headers="keys", tablefmt="grid", showindex=False))
except ImportError:
    print(df_results.to_string(index=False))

## 13. Statistical Significance Tests

In [ ]:
print("\n" + "="*65)
print("  PAIRED STATISTICAL TESTS vs top3_no_tags (baseline)")
print("="*65)

baseline_per_split = [m.get("hr@10", 0) for m in results["top3_no_tags"]["per_split_metrics"]]
sig_rows = []
for name in display_order:
    if name not in results or name == "top3_no_tags":
        continue
    other_per_split = [m.get("hr@10", 0) for m in results[name]["per_split_metrics"]]
    if len(baseline_per_split) > 1 and len(other_per_split) > 1:
        t_stat, p_val = paired_ttest(other_per_split, baseline_per_split)
        sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "ns"
        sig_rows.append({
            "Config": name,
            "Mean Delta hr@10": f"{np.mean(other_per_split) - np.mean(baseline_per_split):+.4f}",
            "t-statistic": f"{t_stat:.3f}",
            "p-value": f"{p_val:.6f}",
            "Significance": sig,
        })

df_sig = pd.DataFrame(sig_rows)
try:
    from tabulate import tabulate
    print(tabulate(df_sig, headers="keys", tablefmt="grid", showindex=False))
except ImportError:
    print(df_sig.to_string(index=False))

## 14. Visualizations

In [ ]:
# Bar chart: hr@10 across all configs
fig, ax = plt.subplots(figsize=(14, 7))
config_names = []
hr_means = []
hr_stds = []
colors = []
color_map = {"no_tags": "#4A90D9", "tags_train_users": "#50C878", "tags_full": "#FFB347"}

for name in display_order:
    if name not in results:
        continue
    r = results[name]
    config_names.append(name)
    hr_means.append(r["avg_metrics"].get("hr@10", 0))
    hr_stds.append(r["std_metrics"].get("hr@10", 0))
    if r.get("shuffle_tags"):
        colors.append("#FF6B6B")
    else:
        colors.append(color_map.get(r["tag_mode"], "#4A90D9"))

bars = ax.bar(range(len(config_names)), hr_means, yerr=hr_stds,
              color=colors, edgecolor="black", linewidth=0.5, capsize=3)
if baseline_hr > 0:
    ax.axhline(y=baseline_hr, color="gray", linestyle="--", alpha=0.7, label=f"Baseline ({baseline_hr:.3f})")
ax.set_xticks(range(len(config_names)))
ax.set_xticklabels(config_names, rotation=45, ha="right", fontsize=9)
ax.set_ylabel("HR@10")
ax.set_title("MARec Cold-Start Performance: Feature Ablation & Tag Modes (10 splits)")
ax.legend()
ax.set_ylim(0, max(hr_means) * 1.15)
for bar, val in zip(bars, hr_means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
            f"{val:.3f}", ha="center", va="bottom", fontsize=8)
plt.tight_layout()
plt.savefig("hr10_comparison.png", dpi=150, bbox_inches="tight")
plt.show()
print("Saved hr10_comparison.png")

In [ ]:
# Per-split variance plot
fig, ax = plt.subplots(figsize=(12, 6))
for name in ["top3_no_tags", "top3_tags_safe", "top3_tags_full"]:
    if name not in results:
        continue
    r = results[name]
    vals = [m.get("hr@10", 0) for m in r["per_split_metrics"]]
    ax.plot(range(len(vals)), vals, "o-", label=name, markersize=6)
ax.set_xlabel("Split Index")
ax.set_ylabel("HR@10")
ax.set_title("Per-Split HR@10 Variance (cold-start stability)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("per_split_variance.png", dpi=150, bbox_inches="tight")
plt.show()
print("Saved per_split_variance.png")

## 15. Save All Results

In [ ]:
output_dir = Path("results/full_experiments")
output_dir.mkdir(parents=True, exist_ok=True)

all_rows = []
for name, r in results.items():
    row = {"config": name, "tag_mode": r["tag_mode"],
           "features": "|".join(r["feature_names"]),
           "shuffled": r.get("shuffle_tags", False)}
    row.update(r["best_hp"])
    row.update({k: v for k, v in r["avg_metrics"].items()})
    row.update({f"{k}_std": v for k, v in r["std_metrics"].items()})
    all_rows.append(row)
df_all = pd.DataFrame(all_rows)
df_all.to_csv(output_dir / "all_results.csv", index=False)
pd.DataFrame([{"config": n, "split": si, **m}
    for n, r in results.items()
    for si, m in enumerate(r["per_split_metrics"])
]).to_csv(output_dir / "per_split_metrics.csv", index=False)

json_results = {}
for name, r in results.items():
    json_results[name] = {
        "config_name": name, "feature_names": r["feature_names"],
        "tag_mode": r["tag_mode"], "best_hp": r["best_hp"],
        "avg_metrics": r["avg_metrics"], "std_metrics": r["std_metrics"],
        "shuffle_tags": r.get("shuffle_tags", False)}
with open(output_dir / "all_results.json", "w") as f:
    json.dump(json_results, f, indent=2, default=float)

print(f"All results saved to {output_dir}/")

## 16. Final Summary

In [ ]:
print("\n" + "="*80)
print("  FINAL EXPERIMENT SUMMARY")
print("="*80)
print(f"\n  Dataset: HetRec 2011 MovieLens")
print(f"  Users: {ds['n_users']}, Items: {ds['n_items']}, Interactions: {ds['URM'].nnz}")
print(f"  Splits: {N_SPLITS}, Cold fraction: 20%")

print(f"\n  Key Results:")
for name in ["top3_no_tags", "top3_tags_safe", "top3_tags_full", "tags_shuffled"]:
    if name in results:
        r = results[name]
        hr = r["avg_metrics"].get("hr@10", 0)
        std = r["std_metrics"].get("hr@10", 0)
        delta = ((hr / baseline_hr) - 1) * 100 if baseline_hr > 0 else 0
        label = "CONTROL" if r.get("shuffle_tags") else "OK"
        print(f"    [{label}] {name:25s} hr@10={hr:.4f}+/-{std:.4f}  ({delta:+.1f}%)")

print(f"\n  Scientific Controls:")
if "tags_shuffled" in results:
    shuf_hr = results["tags_shuffled"]["avg_metrics"].get("hr@10", 0)
    safe_hr = results["top3_tags_safe"]["avg_metrics"].get("hr@10", 0) if "top3_tags_safe" in results else 0
    if shuf_hr < baseline_hr * 1.1:
        print(f"    Shuffled tags collapsed to ~baseline -> tags carry CONTENT signal")
    elif shuf_hr < safe_hr * 0.9:
        print(f"    Shuffled tags much worse than real tags -> content signal confirmed")
    else:
        print(f"    WARNING: Shuffled tags still show improvement -> investigate density effect")

print(f"\n  All {len(results)} experiments completed successfully!")
print("="*80)